In [1]:
import pandas as pd
import numpy as np
import mysql.connector
from datetime import date
from datetime import datetime
from sqlalchemy import create_engine
import sys
import os
from openpyxl import load_workbook

In [2]:
df2022= pd.read_excel("2022 - Activity.xlsx")

In [3]:
df2023=pd.read_excel("2023 - Activity.xlsx")

In [4]:
df2022.columns

Index(['Year', 'Month', 'Date', 'Client', 'Employee ID', 'Status', 'Track',
       'Ceipal_Name', 'Email ID', 'Role', 'Reporting To', 'Team Name', 'Jobs',
       'Submissions', 'Internal Rejections', 'Pending Feedback',
       'Client Submissions', 'Interview Schedules', 'Bulk Mail Sent Count',
       'Bulk Mail Open Count', 'Search String Count', 'Career Builder',
       'Monster', 'Dice', 'Internal DB', 'Total Views', 'G Suite_Total Emails',
       'G Suite_Emails Sent', 'G Suite_Emails Received',
       'Total Inbound No Of Calls ', 'Total Inbound Call Duration',
       '>1 Minute Inbound Call', 'Total No Of OutboundCalls- Connected',
       'Total Outbound Call Duration',
       'Total No Of OutboundCalls-Not Connected Calls',
       '>1 Minute Outbound Call', '>1 Minute Call Outbound Total Time',
       'Avg HT', 'Closure', 'Week', 'Attendance'],
      dtype='object')

In [5]:
excel_to_sql_map = {
    'Employee ID': 'emp_id',
    'Ceipal_Name': 'employee_name',
    'Date': 'date',
    'Client': 'client',
    'Status': 'status',
    'Track': 'track',
    'Email ID': 'email_id',
    'Role': 'role',
    'Reporting To': 'reporting_to',
    'Team Name': 'team_name',
    'Jobs': 'assigned_jobs',
    'Submissions': 'submission',
    'Internal Rejections': 'internal_rejections',
    'Client Submissions': 'client_submissions',
    'Interview Schedules': 'interviewer_schedules',
    'Bulk Mail Sent Count': 'sent_count',
    'Bulk Mail Open Count': 'open_count',
    'Search String Count': 'search_string_count',
    'Career Builder': 'career_builder',
    'Monster': 'monster',
    'Dice': 'dice',
    'Internal DB': 'internal_db',
    'Total Views': 'total_views',
    'Total Inbound No Of Calls ': 'total_inbound_calls',
    'Total Inbound Call Duration': 'total_inbound_duration',
    '>1 Minute Inbound Call': 'inbound_calls_gt_1min',
    'Total No Of OutboundCalls- Connected': 'total_outbound_calls_connected',
    'Total No Of OutboundCalls-Not Connected Calls': 'total_outbound_calls_not_connected',
    'Total Outbound Call Duration': 'total_outbound_duration',
    '>1 Minute Outbound Call': 'outbound_calls_gt_1min',
    '>1 Minute Call Outbound Total Time': 'outbound_duration_gt_1min',
    'Closure': 'closure',
    'Week': 'week',
    'Attendance': 'attendance_type1',
    'Starts': 'starts'  
}


In [6]:
required_order = [
    'emp_id', 'employee_name', 'date', 'client', 'status', 'track', 'email_id',
    'role', 'reporting_to', 'team_name', 'assigned_jobs', 'submission',
    'internal_rejections', 'client_submissions', 'interviewer_schedules',
    'sent_count', 'open_count', 'search_string_count', 'career_builder', 'monster',
    'dice', 'internal_db', 'total_views', 'total_inbound_calls',
    'total_inbound_duration', 'inbound_calls_gt_1min', 'total_outbound_calls_connected',
    'total_outbound_calls_not_connected', 'total_outbound_duration',
    'outbound_calls_gt_1min', 'outbound_duration_gt_1min', 'closure', 'week',
    'attendance_type1', 'starts'  # 'starts' added at the end
]


In [7]:
df2022.columns = df2022.columns.str.strip() 

In [8]:
df2022 = df2022.rename(columns=excel_to_sql_map)

In [9]:
for col in required_order:
    if col not in df2022.columns:
        df2022[col] = None

In [10]:
df2022 = df2022[required_order]


In [11]:
conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Admin@123",
    )
cursor = conn.cursor()
cursor.execute("use All_Data_File")

In [12]:
for col in df2022.columns:
    if pd.api.types.is_datetime64_any_dtype(df2022[col]):
        df2022[col] = df2022[col].dt.strftime('%Y-%m-%d')


In [14]:
for col in df2022.columns:
    if str(df2022[col].dtype).startswith('int') or str(df2022[col].dtype).startswith('float'):
        df2022[col] = df2022[col].fillna(0)
    else:
        df2022[col] = df2022[col].where(pd.notnull(df2022[col]), None)


In [15]:
insert_stmt = (
    "INSERT INTO master_table (" +
    ", ".join(required_order) +
    ") VALUES (" +
    ", ".join(["%s"] * len(required_order)) +
    ")"
)

data_to_insert = df2022.where(pd.notnull(df2022), None).values.tolist()
cursor.executemany(insert_stmt, data_to_insert)
conn.commit()
print(f"{cursor.rowcount} rows inserted successfully using mysql-connector-python!")


124446 rows inserted successfully using mysql-connector-python!


In [16]:
df2023.columns = df2023.columns.str.strip() 

In [17]:
df2023 = df2023.rename(columns=excel_to_sql_map)

In [18]:
for col in required_order:
    if col not in df2023.columns:
        df2023[col] = None

In [19]:
df2023 = df2023[required_order]

In [20]:
for col in df2022.columns:
    if pd.api.types.is_datetime64_any_dtype(df2023[col]):
        df2023[col] = df2023[col].dt.strftime('%Y-%m-%d')


In [24]:
for col in df2023.columns:
    if str(df2023[col].dtype).startswith('int') or str(df2023[col].dtype).startswith('float'):
        df2023[col] = df2023[col].fillna(0)
    else:
        df2023[col] = df2023[col].where(pd.notnull(df2023[col]), None)

In [25]:
insert_stmt = (
    "INSERT INTO master_table (" +
    ", ".join(required_order) +
    ") VALUES (" +
    ", ".join(["%s"] * len(required_order)) +
    ")"
)

data_to_insert = df2023.where(pd.notnull(df2023), None).values.tolist()
cursor.executemany(insert_stmt, data_to_insert)
conn.commit()
print(f"{cursor.rowcount} rows inserted successfully using mysql-connector-python!")


108377 rows inserted successfully using mysql-connector-python!
